<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/ENDS9_Attn_QASC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
python -m spacy download en
python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=52e2189b181aac8233cec65d3e48f108efc98dc31e8497db092c51d8e11cebd0
  Stored in directory: /tmp/pip-ephem-wheel-cache-cwheh5g0/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


**Restart Notebook**

# 3 - Neural Machine Translation by Jointly Learning to Align and Translate

In this third notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473). This model achives our best perplexity yet, ~27 compared to ~34 for the previous model.

## Introduction

As a reminder, here is the general encoder-decoder model:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

In the previous model, our architecture was set-up in a way to reduce "information compression" by explicitly passing the context vector, $z$, to the decoder at every time-step and by passing both the context vector and embedded input word, $d(y_t)$, along with the hidden state, $s_t$, to the linear layer, $f$, to make a prediction.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Even though we have reduced some of this compression, our context vector still needs to contain all of the information about the source sentence. The model implemented in this notebook avoids this compression by allowing the decoder to look at the entire source sentence (via its hidden states) at each decoding step! How does it do this? It uses *attention*. 

Attention works by first, calculating an attention vector, $a$, that is the length of the source sentence. The attention vector has the property that each element is between 0 and 1, and the entire vector sums to 1. We then calculate a weighted sum of our source sentence hidden states, $H$, to get a weighted source vector, $w$. 

$$w = \sum_{i}a_ih_i$$

We calculate a new weighted source vector every time-step when decoding, using it as input to our decoder RNN as well as the linear layer to make a prediction. We'll explain how to do all of this during the session.

## Preparing Data

Again, the preparation is similar to last time.

First we import all the required modules.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd
import random
import math
import time

Set the random seeds for reproducability.

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Load the German and English spaCy models.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [ ]:
!gsutil cp -r \
  gs://unifiedqa/data/qasc/ \
  gs://unifiedqa/data/qasc_test/ \
  gs://unifiedqa/data/qasc_with_ir/ \
  gs://unifiedqa/data/qasc_with_ir_test/ \
  .

Copying gs://unifiedqa/data/qasc/counts.json...
Copying gs://unifiedqa/data/qasc/dev.tsv...
Copying gs://unifiedqa/data/qasc/dev_meta.tsv...
Copying gs://unifiedqa/data/qasc/test.tsv...
/ [4 files][352.9 KiB/352.9 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://unifiedqa/data/qasc/test_meta.tsv...
Copying gs://unifiedqa/data/qasc/train.tsv...
Copying gs://unifiedqa/data/qasc/train_meta.tsv...
Copying gs://unifiedqa/data/qasc_test/counts.json...
Copying gs://unifiedqa/data/qasc_test/dev.tsv...
Copying gs://unifiedqa/data/qasc_test/dev_meta.tsv...
Copying gs://unifiedqa/data/qasc_test/test.tsv...
Copying gs://unifiedqa/data/qasc_test/test_meta.tsv...
Copying gs://unifiedqa/data/qasc_test/train.tsv...
Copying gs://unifiedqa

In [ ]:
import json
sentence = []
label = []
with open('/content/train_qasc.jsonl') as h:
    for line in h:
        example = json.loads(line)
        answerText = [ choice['text'] for choice in example['question']['choices'] if choice['label'] == example['answerKey']][0]
        merged_choices = " [Ans] ".join([choice['text'] for choice in example['question']['choices']])
        combined_q = '[Fact] '+example['combinedfact']+' [Q] '+example['question']['stem']+" [Ans] "+merged_choices
        # scores = []
        # merged_choices = ' A: '.join([choice['text'] for choice in example['question']['choices']])
        # input = 'Q: ' + example['question']['stem'] + ' A: ' + merged_choices
        # correct_answer = [ choice['text'] for choice in example['question']['choices'] if choice['label'] == example['answerKey'] ][0]
        sentence.append(combined_q)
        label.append(answerText)
        # #print(input, correct_answer)
dataset_df2 = pd.DataFrame({'sentence':sentence, 'label':label})

In [ ]:
dataset_df2.sentence[1]

'[Fact] Vapor turning into a liquid leaves behind beads of water [Q] Where do beads of water come from? [Ans] Too much water [Ans] underground systems [Ans] When the water is too cold [Ans] Water spills [Ans] Vapor turning into a liquid [Ans] Warm air moving into cold air [Ans] At the peak of a mountain [Ans] To another location like underground'

In [ ]:
dataset_df.to_csv("qasc.csv",index=False)

In [ ]:
#dataset = pd.read_csv("/content/dev.tsv", sep='\t', header=None)
dataset = pd.read_csv("/content/qasc/train.tsv", sep='\t', header=None)
dataset.columns = ["sentence", "label"]

We create the tokenizers.

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Load the data.

In [ ]:
from torchtext import data 
import matplotlib.pyplot as plt
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =False, include_lengths=False, lower=False)
Label = data.Field(sequential =True, tokenize ='spacy', is_target=False, batch_first =False, include_lengths=False, lower=False)

In [ ]:
dataset = dataset_df2
fields = [('sentence', Sentence),('label',Label)]
example = [data.Example.fromlist([dataset.sentence[i],dataset.label[i]], fields) for i in range(dataset.shape[0])] 
commonqa_ds = data.Dataset(example, fields)
(train, valid) = commonqa_ds.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

Build the vocabulary.

In [ ]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [ ]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  10058
Size of label vocab :  3610
Top 10 words appreared repeatedly : [('[', 69140), (']', 69140), ('Ans', 55312), ('Q', 6915), ('Fact', 6914), ('?', 6156), ('the', 5446), ('of', 4555), ('a', 4463), ('to', 4404)]
Labels :  defaultdict(<function _default_unk_index at 0x7f5ed2817ea0>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'of': 3, 'a': 4, 'and': 5, 'energy': 6, 'water': 7, '.': 8, 'to': 9, 'heat': 10, 'in': 11, 'plants': 12, 'it': 13, 'food': 14, 'animals': 15, 'cells': 16, 'light': 17, 'humans': 18, 'soil': 19, 'air': 20, 'body': 21, 'The': 22, 'A': 23, 'bacteria': 24, "'s": 25, 'Something': 26, 'chemical': 27, 'them': 28, ',': 29, 'an': 30, 'rain': 31, 'wind': 32, '-': 33, 'oxygen': 34, 'It': 35, 'insects': 36, 'on': 37, 'sunlight': 38, 'blood': 39, 'life': 40, 'their': 41, 'system': 42, 'organisms': 43, 'reproduction': 44, 'sun': 45, 'electricity': 46, 'is': 47, 'temperature': 48, 'acid': 49, 'viruses': 50, 'skin': 51, 'trees': 52, 'cancer': 53, 'from': 54, 'pressu

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE=128
train_iterator, test_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False, shuffle=False)

Define the device.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Create the iterators.

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

## Building the Seq2Seq Model

### Encoder

First, we'll build the encoder. Similar to the previous model, we only use a single layer GRU, however we now use a *bidirectional RNN*. With a bidirectional RNN, we have two RNNs in each layer. A *forward RNN* going over the embedded sentence from left to right (shown below in green), and a *backward RNN* going over the embedded sentence from right to left (teal). All we need to do in code is set `bidirectional = True` and then pass the embedded sentence to the RNN as before. 

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq8.png?raw=1)

We now have:

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(e(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(e(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

Where $x_0^\rightarrow = \text{<sos>}, x_1^\rightarrow = \text{guten}$ and $x_0^\leftarrow = \text{<eos>}, x_1^\leftarrow = \text{morgen}$.

As before, we only pass an input (`embedded`) to the RNN, which tells PyTorch to initialize both the forward and backward initial hidden states ($h_0^\rightarrow$ and $h_0^\leftarrow$, respectively) to a tensor of all zeros. We'll also get two context vectors, one from the forward RNN after it has seen the final word in the sentence, $z^\rightarrow=h_T^\rightarrow$, and one from the backward RNN after it has seen the first word in the sentence, $z^\leftarrow=h_T^\leftarrow$.

The RNN returns `outputs` and `hidden`. 

`outputs` is of size **[src len, batch size, hid dim * num directions]** where the first `hid_dim` elements in the third axis are the hidden states from the top layer forward RNN, and the last `hid_dim` elements are hidden states from the top layer backward RNN. We can think of the third axis as being the forward and backward hidden states concatenated together other, i.e. $h_1 = [h_1^\rightarrow; h_{T}^\leftarrow]$, $h_2 = [h_2^\rightarrow; h_{T-1}^\leftarrow]$ and we can denote all encoder hidden states (forward and backwards concatenated together) as $H=\{ h_1, h_2, ..., h_T\}$.

`hidden` is of size **[n layers * num directions, batch size, hid dim]**, where **[-2, :, :]** gives the top layer forward RNN hidden state after the final time-step (i.e. after it has seen the last word in the sentence) and **[-1, :, :]** gives the top layer backward RNN hidden state after the final time-step (i.e. after it has seen the first word in the sentence).

As the decoder is not bidirectional, it only needs a single context vector, $z$, to use as its initial hidden state, $s_0$, and we currently have two, a forward and a backward one ($z^\rightarrow=h_T^\rightarrow$ and $z^\leftarrow=h_T^\leftarrow$, respectively). We solve this by concatenating the two context vectors together, passing them through a linear layer, $g$, and applying the $\tanh$ activation function. 

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

**Note**: this is actually a deviation from the paper. Instead, they feed only the first backward RNN hidden state through a linear layer to get the context vector/decoder initial hidden state. This doesn't seem to make sense to me, so we have changed it.

As we want our model to look back over the whole of the source sentence we return `outputs`, the stacked forward and backward hidden states for every token in the source sentence. We also return `hidden`, which acts as our initial hidden state in the decoder.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

### Attention

Next up is the attention layer. This will take in the previous hidden state of the decoder, $s_{t-1}$, and all of the stacked forward and backward hidden states from the encoder, $H$. The layer will output an attention vector, $a_t$, that is the length of the source sentence, each element is between 0 and 1 and the entire vector sums to 1.

Intuitively, this layer takes what we have decoded so far, $s_{t-1}$, and all of what we have encoded, $H$, to produce a vector, $a_t$, that represents which words in the source sentence we should pay the most attention to in order to correctly predict the next word to decode, $\hat{y}_{t+1}$. 

First, we calculate the *energy* between the previous decoder hidden state and the encoder hidden states. As our encoder hidden states are a sequence of $T$ tensors, and our previous decoder hidden state is a single tensor, the first thing we do is `repeat` the previous decoder hidden state $T$ times. We then calculate the energy, $E_t$, between them by concatenating them together and passing them through a linear layer (`attn`) and a $\tanh$ activation function. 

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

This can be thought of as calculating how well each encoder hidden state "matches" the previous decoder hidden state.

We currently have a **[dec hid dim, src len]** tensor for each example in the batch. We want this to be **[src len]** for each example in the batch as the attention should be over the length of the source sentence. This is achieved by multiplying the `energy` by a **[1, dec hid dim]** tensor, $v$.

$$\hat{a}_t = v E_t$$

We can think of $v$ as the weights for a weighted sum of the energy across all encoder hidden states. These weights tell us how much we should attend to each token in the source sequence. The parameters of $v$ are initialized randomly, but learned with the rest of the model via backpropagation. Note how $v$ is not dependent on time, and the same $v$ is used for each time-step of the decoding. We implement $v$ as a linear layer without a bias.

Finally, we ensure the attention vector fits the constraints of having all elements between 0 and 1 and the vector summing to 1 by passing it through a $\text{softmax}$ layer.

$$a_t = \text{softmax}(\hat{a_t})$$

This gives us the attention over the source sentence!

Graphically, this looks something like below. This is for calculating the very first attention vector, where $s_{t-1} = s_0 = z$. The green/teal blocks represent the hidden states from both the forward and backward RNNs, and the attention computation is all done within the pink block.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq9.png?raw=1)

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

### Decoder

Next up is the decoder. 

The decoder contains the attention layer, `attention`, which takes the previous hidden state, $s_{t-1}$, all of the encoder hidden states, $H$, and returns the attention vector, $a_t$.

We then use this attention vector to create a weighted source vector, $w_t$, denoted by `weighted`, which is a weighted sum of the encoder hidden states, $H$, using $a_t$ as the weights.

$$w_t = a_t H$$

The embedded input word, $d(y_t)$, the weighted source vector, $w_t$, and the previous decoder hidden state, $s_{t-1}$, are then all passed into the decoder RNN, with $d(y_t)$ and $w_t$ being concatenated together.

$$s_t = \text{DecoderGRU}(d(y_t), w_t, s_{t-1})$$

We then pass $d(y_t)$, $w_t$ and $s_t$ through the linear layer, $f$, to make a prediction of the next word in the target sentence, $\hat{y}_{t+1}$. This is done by concatenating them all together.

$$\hat{y}_{t+1} = f(d(y_t), w_t, s_t)$$

The image below shows decoding the first word in an example translation.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq10.png?raw=1)

The green/teal blocks show the forward/backward encoder RNNs which output $H$, the red block shows the context vector, $z = h_T = \tanh(g(h^\rightarrow_T,h^\leftarrow_T)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$, the blue block shows the decoder RNN which outputs $s_t$, the purple block shows the linear layer, $f$, which outputs $\hat{y}_{t+1}$ and the orange block shows the calculation of the weighted sum over $H$ by $a_t$ and outputs $w_t$. Not shown is the calculation of $a_t$.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

### Seq2Seq

This is the first model where we don't have to have the encoder RNN and decoder RNN have the same hidden dimensions, however the encoder has to be bidirectional. This requirement can be removed by changing all occurences of `enc_dim * 2` to `enc_dim * 2 if encoder_is_bidirectional else enc_dim`. 

This seq2seq encapsulator is similar to the last two. The only difference is that the `encoder` returns both the final hidden state (which is the final hidden state from both the forward and backward encoder RNNs passed through a linear layer) to be used as the initial hidden state for the decoder, as well as every hidden state (which are the forward and backward hidden states stacked on top of each other). We also need to ensure that `hidden` and `encoder_outputs` are passed to the decoder. 

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive $z$ and $H$
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and all encoder outputs, $H$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

The rest of this session is very similar to the previous one.

We initialise our parameters, encoder, decoder and seq2seq model (placing it on the GPU if we have one). 

In [ ]:
# INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = len(TRG.vocab)

INPUT_DIM = len(Sentence.vocab)
OUTPUT_DIM = len(Label.vocab)

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10058, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(3610, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=3610, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Calculate the number of parameters. We get an increase of almost 50% in the amount of parameters from the last model. 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 16,405,018 trainable parameters


We create an optimizer.

In [ ]:
optimizer = optim.Adam(model.parameters())

We initialize the loss function.

In [ ]:
# TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

TRG_PAD_IDX = Label.vocab.stoi[Label.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        # src = batch.src
        # trg = batch.trg
        src = batch.sentence
        trg = batch.label
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # src = batch.src
            # trg = batch.trg
            src = batch.sentence
            trg = batch.label

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, define a timing function.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 11s
	Train Loss: 5.937 | Train PPL: 378.819
	 Val. Loss: 7.457 |  Val. PPL: 1731.655
Epoch: 02 | Time: 0m 11s
	Train Loss: 5.896 | Train PPL: 363.682
	 Val. Loss: 7.409 |  Val. PPL: 1651.488
Epoch: 03 | Time: 0m 11s
	Train Loss: 5.860 | Train PPL: 350.609
	 Val. Loss: 7.375 |  Val. PPL: 1595.849
Epoch: 04 | Time: 0m 11s
	Train Loss: 5.828 | Train PPL: 339.581
	 Val. Loss: 7.348 |  Val. PPL: 1552.826
Epoch: 05 | Time: 0m 11s
	Train Loss: 5.794 | Train PPL: 328.339
	 Val. Loss: 7.312 |  Val. PPL: 1498.395
Epoch: 06 | Time: 0m 11s
	Train Loss: 5.761 | Train PPL: 317.707
	 Val. Loss: 7.282 |  Val. PPL: 1454.583
Epoch: 07 | Time: 0m 11s
	Train Loss: 5.727 | Train PPL: 306.935
	 Val. Loss: 7.250 |  Val. PPL: 1407.738
Epoch: 08 | Time: 0m 11s
	Train Loss: 5.703 | Train PPL: 299.784
	 Val. Loss: 7.226 |  Val. PPL: 1375.316
Epoch: 09 | Time: 0m 11s
	Train Loss: 5.693 | Train PPL: 296.777
	 Val. Loss: 7.204 |  Val. PPL: 1344.911
Epoch: 10 | Time: 0m 11s
	Train Loss: 5.689 | 

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.001)

In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 11s
	Train Loss: 5.666 | Train PPL: 288.892
	 Val. Loss: 7.159 |  Val. PPL: 1286.267
Epoch: 02 | Time: 0m 11s
	Train Loss: 5.646 | Train PPL: 283.192
	 Val. Loss: 7.136 |  Val. PPL: 1256.213
Epoch: 03 | Time: 0m 11s
	Train Loss: 5.645 | Train PPL: 282.858
	 Val. Loss: 7.118 |  Val. PPL: 1234.426
Epoch: 04 | Time: 0m 11s
	Train Loss: 5.663 | Train PPL: 287.920
	 Val. Loss: 7.102 |  Val. PPL: 1213.993
Epoch: 05 | Time: 0m 11s
	Train Loss: 5.645 | Train PPL: 282.974
	 Val. Loss: 7.085 |  Val. PPL: 1194.274
Epoch: 06 | Time: 0m 11s
	Train Loss: 5.618 | Train PPL: 275.284
	 Val. Loss: 7.070 |  Val. PPL: 1176.543
Epoch: 07 | Time: 0m 11s
	Train Loss: 5.639 | Train PPL: 281.208
	 Val. Loss: 7.063 |  Val. PPL: 1168.215
Epoch: 08 | Time: 0m 11s
	Train Loss: 5.620 | Train PPL: 275.862
	 Val. Loss: 7.048 |  Val. PPL: 1150.927
Epoch: 09 | Time: 0m 11s
	Train Loss: 5.613 | Train PPL: 273.968
	 Val. Loss: 7.036 |  Val. PPL: 1136.600
Epoch: 10 | Time: 0m 11s
	Train Loss: 5.604 | 

In [ ]:
print(best_valid_loss)

6.957575893402099


In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = 6.957575893402099
model.load_state_dict(torch.load('tut3-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 10s
	Train Loss: 5.592 | Train PPL: 268.302
	 Val. Loss: 6.946 |  Val. PPL: 1038.734
Epoch: 02 | Time: 0m 10s
	Train Loss: 5.577 | Train PPL: 264.269
	 Val. Loss: 6.938 |  Val. PPL: 1030.740
Epoch: 03 | Time: 0m 10s
	Train Loss: 5.566 | Train PPL: 261.334
	 Val. Loss: 6.931 |  Val. PPL: 1023.963
Epoch: 04 | Time: 0m 10s
	Train Loss: 5.581 | Train PPL: 265.449
	 Val. Loss: 6.927 |  Val. PPL: 1019.447
Epoch: 05 | Time: 0m 11s
	Train Loss: 5.574 | Train PPL: 263.515
	 Val. Loss: 6.927 |  Val. PPL: 1018.935
Epoch: 06 | Time: 0m 11s
	Train Loss: 5.574 | Train PPL: 263.576
	 Val. Loss: 6.920 |  Val. PPL: 1012.682
Epoch: 07 | Time: 0m 11s
	Train Loss: 5.564 | Train PPL: 260.952
	 Val. Loss: 6.921 |  Val. PPL: 1013.050
Epoch: 08 | Time: 0m 11s
	Train Loss: 5.551 | Train PPL: 257.491
	 Val. Loss: 6.916 |  Val. PPL: 1007.974
Epoch: 09 | Time: 0m 11s
	Train Loss: 5.551 | Train PPL: 257.557
	 Val. Loss: 6.924 |  Val. PPL: 1016.033
Epoch: 10 | Time: 0m 11s
	Train Loss: 5.541 | 

In [ ]:
print(best_valid_loss)

6.915698003768921


In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = 6.915698003768921
model.load_state_dict(torch.load('tut3-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 11s
	Train Loss: 5.546 | Train PPL: 256.124
	 Val. Loss: 6.914 |  Val. PPL: 1006.667
Epoch: 02 | Time: 0m 11s
	Train Loss: 5.550 | Train PPL: 257.222
	 Val. Loss: 6.911 |  Val. PPL: 1003.350
Epoch: 03 | Time: 0m 11s
	Train Loss: 5.551 | Train PPL: 257.554
	 Val. Loss: 6.914 |  Val. PPL: 1005.885
Epoch: 04 | Time: 0m 11s
	Train Loss: 5.550 | Train PPL: 257.274
	 Val. Loss: 6.910 |  Val. PPL: 1001.761
Epoch: 05 | Time: 0m 11s
	Train Loss: 5.544 | Train PPL: 255.800
	 Val. Loss: 6.908 |  Val. PPL: 1000.417
Epoch: 06 | Time: 0m 11s
	Train Loss: 5.535 | Train PPL: 253.482
	 Val. Loss: 6.907 |  Val. PPL: 999.556
Epoch: 07 | Time: 0m 11s
	Train Loss: 5.525 | Train PPL: 250.780
	 Val. Loss: 6.911 |  Val. PPL: 1003.377
Epoch: 08 | Time: 0m 11s
	Train Loss: 5.529 | Train PPL: 251.919
	 Val. Loss: 6.912 |  Val. PPL: 1004.590
Epoch: 09 | Time: 0m 11s
	Train Loss: 5.518 | Train PPL: 249.178
	 Val. Loss: 6.918 |  Val. PPL: 1010.431
Epoch: 10 | Time: 0m 11s
	Train Loss: 5.506 | T

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.000005, weight_decay=0.001)
#optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.001)
print(best_valid_loss)

6.885243463516235


In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = 6.885243463516235
model.load_state_dict(torch.load('tut3-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 11s
	Train Loss: 5.411 | Train PPL: 223.833
	 Val. Loss: 6.883 |  Val. PPL: 975.389
Epoch: 02 | Time: 0m 11s
	Train Loss: 5.399 | Train PPL: 221.153
	 Val. Loss: 6.882 |  Val. PPL: 975.024
Epoch: 03 | Time: 0m 11s
	Train Loss: 5.416 | Train PPL: 224.874
	 Val. Loss: 6.884 |  Val. PPL: 976.866
Epoch: 04 | Time: 0m 11s
	Train Loss: 5.421 | Train PPL: 225.997
	 Val. Loss: 6.883 |  Val. PPL: 975.817
Epoch: 05 | Time: 0m 11s
	Train Loss: 5.409 | Train PPL: 223.324
	 Val. Loss: 6.877 |  Val. PPL: 969.386
Epoch: 06 | Time: 0m 11s
	Train Loss: 5.411 | Train PPL: 223.948
	 Val. Loss: 6.876 |  Val. PPL: 968.954
Epoch: 07 | Time: 0m 11s
	Train Loss: 5.413 | Train PPL: 224.290
	 Val. Loss: 6.878 |  Val. PPL: 970.891
Epoch: 08 | Time: 0m 11s
	Train Loss: 5.390 | Train PPL: 219.127
	 Val. Loss: 6.875 |  Val. PPL: 967.853
Epoch: 09 | Time: 0m 11s
	Train Loss: 5.403 | Train PPL: 221.966
	 Val. Loss: 6.881 |  Val. PPL: 973.979
Epoch: 10 | Time: 0m 11s
	Train Loss: 5.394 | Train PPL

In [ ]:
best_valid_loss

6.870632219314575

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.000001, weight_decay=0.01)

N_EPOCHS = 20
CLIP = 1

best_valid_loss = 6.870632219314575
model.load_state_dict(torch.load('tut3-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 11s
	Train Loss: 5.392 | Train PPL: 219.688
	 Val. Loss: 6.870 |  Val. PPL: 963.373
Epoch: 02 | Time: 0m 11s
	Train Loss: 5.380 | Train PPL: 217.111
	 Val. Loss: 6.871 |  Val. PPL: 964.100
Epoch: 03 | Time: 0m 11s
	Train Loss: 5.388 | Train PPL: 218.733
	 Val. Loss: 6.870 |  Val. PPL: 962.517
Epoch: 04 | Time: 0m 11s
	Train Loss: 5.397 | Train PPL: 220.740
	 Val. Loss: 6.870 |  Val. PPL: 963.158
Epoch: 05 | Time: 0m 11s
	Train Loss: 5.395 | Train PPL: 220.309
	 Val. Loss: 6.868 |  Val. PPL: 960.636
Epoch: 06 | Time: 0m 11s
	Train Loss: 5.400 | Train PPL: 221.386
	 Val. Loss: 6.868 |  Val. PPL: 960.646
Epoch: 07 | Time: 0m 11s
	Train Loss: 5.391 | Train PPL: 219.425
	 Val. Loss: 6.866 |  Val. PPL: 958.905
Epoch: 08 | Time: 0m 11s
	Train Loss: 5.402 | Train PPL: 221.851
	 Val. Loss: 6.865 |  Val. PPL: 958.142
Epoch: 09 | Time: 0m 11s
	Train Loss: 5.405 | Train PPL: 222.477
	 Val. Loss: 6.864 |  Val. PPL: 957.659
Epoch: 10 | Time: 0m 11s
	Train Loss: 5.398 | Train PPL

In [ ]:
#optimizer = optim.Adam(model.parameters(), lr=0.000001, weight_decay=0.01)

N_EPOCHS = 50
CLIP = 1

#best_valid_loss = 6.870632219314575
model.load_state_dict(torch.load('tut3-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 11s
	Train Loss: 5.424 | Train PPL: 226.770
	 Val. Loss: 6.850 |  Val. PPL: 943.501
Epoch: 02 | Time: 0m 11s
	Train Loss: 5.432 | Train PPL: 228.597
	 Val. Loss: 6.848 |  Val. PPL: 941.946
Epoch: 03 | Time: 0m 11s
	Train Loss: 5.448 | Train PPL: 232.406
	 Val. Loss: 6.850 |  Val. PPL: 944.287
Epoch: 04 | Time: 0m 11s
	Train Loss: 5.438 | Train PPL: 229.927
	 Val. Loss: 6.850 |  Val. PPL: 943.495
Epoch: 05 | Time: 0m 11s
	Train Loss: 5.425 | Train PPL: 227.049
	 Val. Loss: 6.847 |  Val. PPL: 941.157
Epoch: 06 | Time: 0m 11s
	Train Loss: 5.432 | Train PPL: 228.605
	 Val. Loss: 6.847 |  Val. PPL: 940.789
Epoch: 07 | Time: 0m 11s
	Train Loss: 5.433 | Train PPL: 228.774
	 Val. Loss: 6.846 |  Val. PPL: 939.988
Epoch: 08 | Time: 0m 11s
	Train Loss: 5.443 | Train PPL: 231.143
	 Val. Loss: 6.847 |  Val. PPL: 940.754
Epoch: 09 | Time: 0m 11s
	Train Loss: 5.439 | Train PPL: 230.154
	 Val. Loss: 6.846 |  Val. PPL: 940.374
Epoch: 10 | Time: 0m 11s
	Train Loss: 5.440 | Train PPL

In [ ]:
#optimizer = optim.Adam(model.parameters(), lr=0.000001, weight_decay=0.01)

N_EPOCHS = 50
CLIP = 1

#best_valid_loss = 6.870632219314575
model.load_state_dict(torch.load('tut3-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 11s
	Train Loss: 5.496 | Train PPL: 243.654
	 Val. Loss: 6.826 |  Val. PPL: 921.856
Epoch: 02 | Time: 0m 11s
	Train Loss: 5.505 | Train PPL: 245.936
	 Val. Loss: 6.828 |  Val. PPL: 922.972
Epoch: 03 | Time: 0m 11s
	Train Loss: 5.498 | Train PPL: 244.257
	 Val. Loss: 6.828 |  Val. PPL: 923.124
Epoch: 04 | Time: 0m 11s
	Train Loss: 5.491 | Train PPL: 242.595
	 Val. Loss: 6.829 |  Val. PPL: 924.382
Epoch: 05 | Time: 0m 11s
	Train Loss: 5.497 | Train PPL: 243.995
	 Val. Loss: 6.826 |  Val. PPL: 921.371
Epoch: 06 | Time: 0m 11s
	Train Loss: 5.505 | Train PPL: 245.851
	 Val. Loss: 6.827 |  Val. PPL: 922.407
Epoch: 07 | Time: 0m 11s
	Train Loss: 5.504 | Train PPL: 245.684
	 Val. Loss: 6.826 |  Val. PPL: 921.284
Epoch: 08 | Time: 0m 11s
	Train Loss: 5.514 | Train PPL: 248.223
	 Val. Loss: 6.828 |  Val. PPL: 923.020
Epoch: 09 | Time: 0m 11s
	Train Loss: 5.510 | Train PPL: 247.085
	 Val. Loss: 6.828 |  Val. PPL: 923.781
Epoch: 10 | Time: 0m 11s
	Train Loss: 5.497 | Train PPL

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0000001, weight_decay=0.05)
print(best_valid_loss)

6.825387096405029


In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = 6.825387096405029
model.load_state_dict(torch.load('tut3-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 11s
	Train Loss: 5.515 | Train PPL: 248.360
	 Val. Loss: 6.824 |  Val. PPL: 919.615
Epoch: 02 | Time: 0m 11s
	Train Loss: 5.505 | Train PPL: 245.929
	 Val. Loss: 6.823 |  Val. PPL: 918.536
Epoch: 03 | Time: 0m 11s
	Train Loss: 5.511 | Train PPL: 247.358
	 Val. Loss: 6.822 |  Val. PPL: 917.361
Epoch: 04 | Time: 0m 11s
	Train Loss: 5.514 | Train PPL: 248.263
	 Val. Loss: 6.820 |  Val. PPL: 916.392
Epoch: 05 | Time: 0m 11s
	Train Loss: 5.499 | Train PPL: 244.494
	 Val. Loss: 6.819 |  Val. PPL: 915.384
Epoch: 06 | Time: 0m 11s
	Train Loss: 5.519 | Train PPL: 249.412
	 Val. Loss: 6.818 |  Val. PPL: 914.577
Epoch: 07 | Time: 0m 11s
	Train Loss: 5.509 | Train PPL: 247.006
	 Val. Loss: 6.817 |  Val. PPL: 913.611
Epoch: 08 | Time: 0m 11s
	Train Loss: 5.508 | Train PPL: 246.695
	 Val. Loss: 6.817 |  Val. PPL: 912.981
Epoch: 09 | Time: 0m 11s
	Train Loss: 5.512 | Train PPL: 247.582
	 Val. Loss: 6.816 |  Val. PPL: 912.267
Epoch: 10 | Time: 0m 11s
	Train Loss: 5.503 | Train PPL

In [ ]:
print(best_valid_loss)

6.695293951034546


In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = 6.695293951034546
model.load_state_dict(torch.load('tut3-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 9s
	Train Loss: 5.408 | Train PPL: 223.193
	 Val. Loss: 6.648 |  Val. PPL: 771.149
Epoch: 02 | Time: 0m 8s
	Train Loss: 5.525 | Train PPL: 250.857
	 Val. Loss: 6.630 |  Val. PPL: 757.622
Epoch: 03 | Time: 0m 9s
	Train Loss: 5.668 | Train PPL: 289.429
	 Val. Loss: 6.644 |  Val. PPL: 768.028
Epoch: 04 | Time: 0m 9s
	Train Loss: 5.779 | Train PPL: 323.584
	 Val. Loss: 6.664 |  Val. PPL: 783.420
Epoch: 05 | Time: 0m 9s
	Train Loss: 5.901 | Train PPL: 365.547
	 Val. Loss: 6.684 |  Val. PPL: 799.416
Epoch: 06 | Time: 0m 9s
	Train Loss: 5.987 | Train PPL: 398.399
	 Val. Loss: 6.694 |  Val. PPL: 807.332
Epoch: 07 | Time: 0m 9s
	Train Loss: 6.036 | Train PPL: 418.334
	 Val. Loss: 6.702 |  Val. PPL: 813.658
Epoch: 08 | Time: 0m 9s
	Train Loss: 6.067 | Train PPL: 431.425
	 Val. Loss: 6.709 |  Val. PPL: 819.505
Epoch: 09 | Time: 0m 9s
	Train Loss: 6.103 | Train PPL: 447.194
	 Val. Loss: 6.714 |  Val. PPL: 823.743
Epoch: 10 | Time: 0m 9s
	Train Loss: 6.130 | Train PPL: 459.558


In [ ]:
!cp /content/QASC_Attn.pt /content/drive/MyDrive/EVA4/ENDS9

Finally, we test the model on the test set using these "best" parameters.

In [ ]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.178 | Test PPL:  24.009 |


We've improved on the previous model, but this came at the cost of doubling the training time.

Next, we'll be using the same architecture but using a few tricks that are applicable to all RNN architectures - packed padded sequences and masking. We'll also implement code which will allow us to look at what words in the input the RNN is paying attention to when decoding the output. Check this [Notebook](https://colab.research.google.com/github/bentrevett/pytorch-seq2seq/blob/master/4%20-%20Packed%20Padded%20Sequences%2C%20Masking%2C%20Inference%20and%20BLEU.ipynb) 